In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

material:
    type: Text
    value: Ni
    maxlen: 2
    options: [Ni, Al, Fe, Co, Ti, W]
    
crystal_structure:
    type: Text
    value: fcc
    options: [fcc, bcc, hcp]

lattice_parameter:
    type: Number
    value: 3.5203
    min: 3
    max: 5.5
    units: angstroms
    
mass:
    type: Element
    property: atomic_weight
    value: Ni

Tsolid:
    type: Number
    value: 500
    min: 1
    max: 4000
    units: K
    
Tliquid:
    type: Number
    value: 2000
    min: 1
    max: 4000
    units: K
    
kim_model_name:
    type: Text
    value: "EAM_Dynamo_PurjaPun_Mishin_NiAl__MO_751354403791_004"
    

In [ ]:
%%yaml OUTPUTS

final_snapshot:
    desc: JPG of final structure
    type: Image

melting_temperature:
    desc: The melting temperature
    type: Number
    units: K

LAMMPS_log:
    desc: Lammps log file
    type: Text

time_series:
    desc: Time
    type: Array
    units: ps

temperature_series:
    desc: Temperature
    type: Array
    units: K
    
potential_energy_series:
    type: Array
    units: eV

volume_series:
    type: Array
    units: angstrom^3
    

In [ ]:
# PARAMETERS (for testing)
# These get replaced when SimTool is run
material = 'Ni'
crystal_structure = 'fcc'
lattice_parameter = 3.5203
mass = 58.6934
Tsolid = 2000
Tliquid = 500
kim_model_name = "EAM_Dynamo_PurjaPun_Mishin_NiAl__MO_751354403791_004"

In [ ]:
# put inputs in a dict for easy substitution with python format
inputs = {
    'crystal_structure': crystal_structure,
    'lattice_parameter': lattice_parameter,
    'mass': mass,
    'KIM_model_name': kim_model_name,
    'material': material,
    'Tsolid': Tsolid,
    'Tliquid': Tliquid
}

In [ ]:
#Build LAMMPS input file
lammps_name = 'melt-%s.in' % material
logname = 'log-%s.lammps' % material

inputfile = '''
    #LAMMPS input file
    boundary        p p p
    units           metal

    lattice         {crystal_structure} {lattice_parameter} orient x 1 0 0 orient y 0 1 0 orient z 0 0 1 origin 0.01 0.01 0.01
    region          box2 block 0 18 0 8 0 8
    create_box      1 box2
    create_atoms    1 box
    mass            * {mass}

    region          Rsolid block 0 9 INF INF INF INF
    group           Gsolid region Rsolid
    region          Rliquid block 9 18 INF INF INF INF
    group           Gliquid region Rliquid

    pair_style      kim KIMvirial {KIM_model_name}
    pair_coeff      * * {material}

    compute         Tsolid Gsolid temp
    compute         Tliquid Gliquid temp

    thermo          100
    thermo_style    custom step etotal pe ke temp c_Tsolid c_Tliquid press vol pxx pyy pzz pxy pxz pyz lx ly lz
    timestep        0.001
    neighbor        1.0 bin
    neigh_modify    once no every 1 delay 0 check yes

    minimize        1e-6 1e-10 10000 100000

    dump            d1 all image 20000 main.*.ppm type type view 90 90
    velocity        Gsolid create {Tsolid} 123456 dist gaussian mom yes rot yes
    velocity        Gliquid create {Tliquid} 123456 dist gaussian mom yes rot yes

    fix             f3 Gsolid npt temp {Tsolid} {Tsolid} 0.1 aniso 1.01 1.01 1
    fix             f4 Gliquid npt temp {Tliquid} {Tliquid} 0.1 aniso 1.01 1.01 1
    run             10000
    unfix           f3
    unfix           f4
    #write_data      intermediate.data

    fix             f1 all nph aniso 1.01 1.01 1
    run             50000
    #write_data      nph.data
'''.format(**inputs)

with open(lammps_name, "w") as f:
    f.write(inputfile)

In [ ]:
write_string = '''#!/bin/sh
lammpsInput=$1
kimModel=$2
logname=$3

. /etc/environ.sh

if [ -n "${ANACONDA_CHOICE}" ] ; then
   unuse -e ${ANACONDA_CHOICE}
fi

use -e -r openkim-1.6.3
use -e -r lammps-22Aug18

downloadkim.sh ${kimModel}

lmp_serial -in ${lammpsInput} -l ${logname}
'''
with open("runlammps.sh", "w") as f:
    f.write(write_string)

In [ ]:
!bash runlammps.sh {lammps_name} {kim_model_name} {logname}

In [ ]:
# import lammps reader (Python 3 version)
import sys
sys.path.append("/apps/share64/debian7/lammps/lammps-22Aug18/tools/python/pizza")
from log3 import log

In [ ]:
logfile = log(logname)
time, temperature, potential_energy, volume = logfile.get("Step", "Temp", "PotEng", "Volume")

import numpy as np
time = np.array(time)
temperature = np.array(temperature)

idx = np.where(time >= 3e4)
melting_temperature = np.average(temperature[idx[0]])

In [ ]:
#----------- OUTPUTS ---------#
'''
Here's a list of outputs:
1. Melting temperature (units: K)
2. Arrays for time, temperature, potential_energy and volume (units: ps, K, eV, Å^3 respectively)
3. JPG of final structure (filename: 'final.jpg')
'''
print(melting_temperature)
!convert main.20000.ppm main.20000.jpg
!mv main.20000.jpg final.jpg

In [ ]:
from IPython.display import Image
from hublib.tool import save
save('final_snapshot', Image('final.jpg'))

In [ ]:
save('melting_temperature', melting_temperature)
with open(logname) as fp:
    save('LAMMPS_log', fp.read())

In [ ]:
save('time_series', time)
save('temperature_series', temperature)
save('potential_energy_series', potential_energy)
save('volume_series', volume)